In [ ]:
import sys

# setting path
sys.path.append('..')

import numpy as np
import pandas as pd
import time

from src.models.bocs.LinReg import LinReg
from src.models.GPr import GPr
from ngboost import NGBRegressor
from ngboost.distns import LogNormal, Normal, Exponential
from ngboost.scores import CRPS, LogScore
#from xgboost import XGBRegressor

# Learners
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor

from src.problems.contamination import Contamination
from src.problems.rna import pRNA
from src.problems.bqp import BQP
from src.problems.latin_square import LatinSquare

from src.sbbo_mh import MHSBBO
from src.sbbo_gibbs import GibbsSBBO

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

from src.uncertainty_metrics import *


# Convergence

In [ ]:
opt_prob = Contamination(n=100, lamda=0.0001, random_seed=305)

In [ ]:
N=39 # Number of repetitions
params = {}
params["cooling_schedule"] = np.arange(100, 500, 10)
params["burnin"] = 0.8
params["modal"]  = True
params["af"] = "EI"

learner = GPr()
params["model"] = learner

search = MHSBBO(opt_prob, params)

df_def = search.compute_trace()
df_def["Exp"] = 0

for i in range(N):
    
    params = {}
    params["cooling_schedule"] = np.arange(100, 1000, 10)
    params["burnin"] = 0.8
    params["modal"]  = True
    params["af"] = "EI"

    learner = GPr()
    params["model"] = learner

    search = MHSBBO(opt_prob, params)

    df = search.compute_trace()
    df["Exp"] = i+1
    df_def = pd.concat([df_def, df])

In [ ]:
df_def

In [ ]:
df_def.to_csv("convergence.csv", index=False)